In [1]:
import numpy as np
import datacube
from datacube.api import GridWorkflow
from madmex.wrappers import gwf_query
import fiona
import pprint
from madmex.io.vector_db import VectorDb, load_segmentation_from_dataset
from madmex.models import PredictObject
from madmex.models import PredictClassification, Tag, Region
from django.contrib.gis.geos import Polygon
from madmex.util import chunk
import json
from shapely.geometry import shape, mapping
from madmex.util.spatial import geometry_transform
from affine import Affine
from rasterio.features import rasterize
import rasterio

In [2]:
#name of prediction

In [2]:
name_predict = 's2_fiona_predict'
predict_object_id = 1 #TODO: delete this and change it to a programatically way

In [3]:
pred_objects = PredictClassification.objects.filter(predict_object_id=predict_object_id,name=name_predict)

In [4]:
len(pred_objects)

10000

In [5]:
seg = PredictObject.objects.filter(id=pred_objects[0].predict_object_id)

In [6]:
path = seg[0].path

In [10]:
#intersect with region Jalisco

In [7]:
region = 'Jalisco'

In [8]:
region_geom = Region.objects.get(name=region).the_geom

In [9]:
region_geojson = region_geom.geojson

In [10]:
geometry = json.loads(region_geojson)

In [11]:
proj4='+proj=lcc +lat_1=17.5 +lat_2=29.5 +lat_0=12 +lon_0=-102 +x_0=2500000 +y_0=0 +a=6378137 +b=6378136.027241431 +units=m +no_defs'

In [12]:
geometry_proj = geometry_transform(geometry,proj4)

In [13]:
shape_Jalisco=shape(geometry_proj)

In [14]:
with fiona.open(path) as src:
    source_driver = src.driver
    source_crs = src.crs
    source_schema = src.schema
    pprint.pprint(src[0])

{'geometry': {'coordinates': [[(2427740.0, 1026160.0),
                               (2428250.0, 1026160.0),
                               (2428250.0, 1025650.0),
                               (2427740.0, 1025650.0),
                               (2427740.0, 1026160.0)]],
              'type': 'Polygon'},
 'id': '0',
 'properties': OrderedDict([('id', 0)]),
 'type': 'Feature'}


In [15]:
qs = Tag.objects.filter(scheme='madmex_31')
tag_mapping = {x.id:x.numeric_code for x in qs}
tag_id_list = [pred_objects[x].tag_id for x in range(0,len(pred_objects))]

In [16]:
labels=[tag_mapping[tag_id_list[x]] for x in range(0,len(tag_id_list))]

In [17]:
fc_schema = {'geometry': 'Polygon',
             'properties': {'code': 'int'}}

In [18]:
with fiona.open(path) as src:
    fc_pred = [(x[0]['geometry'], x[1]) for x in zip(src, labels)]

In [19]:
fc_pred[0][0]

{'type': 'Polygon',
 'coordinates': [[(2427740.0, 1026160.0),
   (2428250.0, 1026160.0),
   (2428250.0, 1025650.0),
   (2427740.0, 1025650.0),
   (2427740.0, 1026160.0)]]}

In [20]:
bbox = seg[0].the_geom

In [21]:
xmin, ymin, xmax, ymax = bbox.extent
resolution = 10.0

In [22]:
# Define output raster shape
nrows = int(((ymax - ymin) // resolution) + 1)
ncols = int(((xmax - xmin) // resolution) + 1)
shape_dim = (nrows, ncols)

In [23]:
arr = np.zeros((nrows, ncols), dtype=np.uint8)

In [24]:
aff = Affine(resolution, 0, xmin, 0, -resolution, ymax)

In [25]:
aff

Affine(10.0, 0.0, 2427740.0,
       0.0, -10.0, 1026160.0)

In [26]:
geometry_label_intersect = tuple()

In [27]:
geometry_label_intersect = [(mapping(shape(feat[0]).intersection(shape_Jalisco)),
                                     feat[1]) for feat in fc_pred if shape(feat[0]).intersects(shape_Jalisco)]

In [28]:
len(geometry_label_intersect)

10000

In [29]:
geometry_label_intersect[0:2]

[({'type': 'Polygon',
   'coordinates': (((2427740.0, 1026160.0),
     (2428250.0, 1026160.0),
     (2428250.0, 1025650.0),
     (2427740.0, 1025650.0),
     (2427740.0, 1026160.0)),)},
  28),
 ({'type': 'Polygon',
   'coordinates': (((2428250.0, 1026160.0),
     (2428750.0, 1026160.0),
     (2428750.0, 1025650.0),
     (2428250.0, 1025650.0),
     (2428250.0, 1026160.0)),)},
  11)]

In [30]:
rasterize(shapes=geometry_label_intersect, transform=aff, dtype=np.uint8, out=arr)

array([[28, 28, 28, ..., 27, 27,  0],
       [28, 28, 28, ..., 27, 27,  0],
       [28, 28, 28, ..., 27, 27,  0],
       ...,
       [28, 28, 28, ..., 28, 28,  0],
       [28, 28, 28, ..., 28, 28,  0],
       [ 0,  0,  0, ...,  0,  0,  0]], dtype=uint8)

In [31]:
meta = {'driver': 'GTiff',
         'width': shape_dim[1],
         'height': shape_dim[0],
         'count': 1,
         'dtype': arr.dtype,
         'crs': proj4,
         'transform': aff,
         'compress': 'lzw',
        'nodata': 0}

In [32]:
filename='/shared_volume/tasks/2019/fiona_cloud_usage/classif_and_intersection_fiona.tif'

In [33]:
with rasterio.open(filename, 'w', **meta) as dst:
    dst.write(arr, 1)